# dataframe 1

In [1]:
import pandas as pd
import re
from Bio.SeqIO.FastaIO import SimpleFastaParser
import gzip

# Function to return the count number  in the fasta in a dictionary
def lncRNA_db(file_path, separator, RNA_index):
    lncRNA_dictionary = {}
    with gzip.open(file_path, 'r') as fasta_file:
        fasta_contents = fasta_file.readlines()
        for line in fasta_contents:
            if line.startswith(">"):
                name = line.split(separator)[RNA_index]
                if name not in lncRNA_dictionary:
                    lncRNA_dictionary[name] = 1
                else:
                    lncRNA_dictionary[name] += 1
    return lncRNA_dictionary

# Function to read fasta and return a DataFrame
def read_fasta(file_path, columns):
    file_path = "C:/Users/Swadha/Desktop/INTERNSHIP 2024/data collection/lncRNA_LncBookv2.0.fa.gz"
   
    bd_func = lncRNA_db(file_path, '|', 1)  # Execute function
    
    with gzip.open(file_path, 'rt') as fasta_file:
        records = []  # Create empty list
        for title, sequence in SimpleFastaParser(fasta_file):
            record = []
            title_splits = re.findall(r"[\w']+", title)  # Data cleaning
            
            record.append(title_splits[0])  # First values are ID
            record.append(len(sequence))  # Second values are sequence lengths
            sequence = " ".join(sequence)  # It converts into one line
            record.append(sequence)  # Third values are sequences

            
            records.append(record)  # Add the record to the list

    return pd.DataFrame(records, columns=columns)  # Return DataFrame

# Example usage
file_path = "lncRNA_LncBookv2.0.fa.gz"
columns = ["id", "sequence_length", "sequence"]
data = read_fasta(file_path, columns)

print(data)


TypeError: startswith first arg must be bytes or a tuple of bytes, not str

# dataframe 2

In [ ]:
import pandas as pd
import re
from Bio import SeqIO
import gzip

# Function to return the count number of RNA types in the FASTA file
def lncRNA_db(file_path, separator, RNA_index):
    lncRNA_dictionary = {}
    with gzip.open(file_path, 'rt') as fasta_file:
        for line in fasta_file:
            if line.startswith(">"):
                name = line.split(separator)[RNA_index]
                if name not in lncRNA_dictionary:
                    lncRNA_dictionary[name] = 1
                else:
                    lncRNA_dictionary[name] += 1
    return lncRNA_dictionary

# Function to read FASTA file and return a DataFrame
def read_fasta(file_path, columns):
    # Get the RNA count dictionary
    bd_func = lncRNA_db(file_path, '|', 1)  # Execute function
    
    records = []  # Create empty list to store records
    with gzip.open(file_path, 'rt') as handle:
        for record in SeqIO.parse(handle, 'fasta'):
            rec = []
            title_splits = re.findall(r"[\w']+", record.description)  # Data cleaning
            
            rec.append(title_splits[0])  # First value is ID
            rec.append(len(record.seq))  # Second value is sequence length
            rec.append(str(record.seq))  # Third value is the sequence
            
            records.append(rec)  # Add the record to the list

    # Return DataFrame and RNA count dictionary
    return pd.DataFrame(records, columns=columns), bd_func

# Example usage
file_path = "lncRNA_LncBookv2.0.fa.gz"
columns = ["id", "sequence_length", "sequence"]
data, rna_counts = read_fasta(file_path, columns)

print(data)
print(rna_counts)


# dataframe 3

In [ ]:
import pandas as pd
import re
import gzip
# a function to return the count number of animals in the fasta in a dictionary
def lncRNA_db(file_path, separator, index):
  lncRNA_dictionary={}
  with gzip.open("lncRNA_LncBookv2.0.fa.gz") as fasta_file:
    fasta_contents = fasta_file.readlines()
    for line in fasta_contents:
      if line.startswith(">"):
        name = line.split(separator)[index] 
        if name not in lncRNA_dictionary:
          lncRNA_dictionary[name] = 1
        else:
          lncRNA_dictionary[name] = lncRNA_dictionary[name] + 1
  
  return lncRNA_dictionary


def read_fasta(file_path, columns) :
    bd_func = lncRNA_db(file_path, '|', 1) # execute function
 
  from Bio.SeqIO.FastaIO import SimpleFastaParser 
  with gzip.open("lncRNA_LncBookv2.0.fa.gz") as fasta_file :
    records = [] # create empty list
    for title, sequence in SimpleFastaParser(fasta_file): #SimpleFastaParser Iterate over Fasta records as string tuples. For each record a tuple of two strings is returned, the FASTA title line (without the leading ‘>’ character), and the sequence (with any whitespace removed). 
      record = []
      title_splits=re.findall(r"[\w']+", title) # Data cleaning is needed

      record.append(title_splits[0])  #First values are ID (Append adds element to a list)
      record.append(len(sequence)) #Second values are sequences lengths
      sequence = " ".join(sequence) #It converts into one line
      record.append(sequence)#Third values are sequences
    
   
    # print(records)
pd.DataFrame(record, columns = columns) #We have created a function that returns a dataframe

#Now let's use this function by inserting in the first argument the file name (or file path if your working directory is different from where the fasta file is)    
#And in the second one the names of columns
data= read_fasta("lncRNA_LncBookv2.0.fa.gz", columns=["id","sequence_length", "sequence"])

print(data)